In [1]:
from openbabel import pybel
from openbabel import openbabel as ob
# from openbabel import _openbabel as ob_
from rdkit import Chem
import subprocess
import os
import re

AMBERHOME = "/Users/suyian/opt/anaconda3/envs/autosolvate"
current_dir = os.getcwd()

MolID194 = '194-1' # Cr(CN)6 3-
MolMetalName194 = 'CR'
MolCharge194 = -3
MolMetalValence194 = 3

MolID239 = '239-1' # Cr(H2O)6 3+
MolMetalName239 = 'CR'
MolCharge239 = 3
MolMetalValence239 = 3

In [2]:
# 文件类定义

class Mol2_file:

    class Atom:
        def __init__(self, atom_id, atom_name, x, y, z, atom_type, subst_id, subst_name, charge):
            self.atom_id = atom_id
            self.atom_name = atom_name
            self.x = x
            self.y = y
            self.z = z
            self.atom_type = atom_type
            self.subst_id = subst_id
            self.subst_name = subst_name
            self.charge = charge

    class Bond:
        def __init__(self, bond_id, origin_atom_id, target_atom_id, bond_type):
            self.bond_id = bond_id
            self.origin_atom_id = origin_atom_id
            self.target_atom_id = target_atom_id
            self.bond_type = bond_type


    def __init__(self, content):
        self.molecule_info = {}
        self.atoms = []
        self.bonds = []
        self.parse_content(content)
    
    def parse_content(self, content):
        sections = content.split("@<TRIPOS>")
        for section in sections:
            lines = section.strip().split("\n")
            if "MOLECULE" == lines[0].strip('@<TRIPOS>'):
                self.parse_molecule_info(lines[1:])
            elif "ATOM" == lines[0].strip('@<TRIPOS>'):
                self.parse_atoms(lines[1:])
            elif "BOND" == lines[0].strip('@<TRIPOS>'):
                self.parse_bonds(lines[1:])
                
    def parse_molecule_info(self, lines):
        self.molecule_info["name"] = lines[0].strip()
        self.molecule_info["num_atoms"], self.molecule_info["num_bonds"], *rest = map(int, lines[1].split())
        self.molecule_info["type"] = lines[2].strip()
        self.molecule_info["charge_method"] = lines[3].strip()
        
    def parse_atoms(self, lines):
        for line in lines:
            parts = re.split(r'\s+', line.strip())
            atom_id = int(parts[0])
            atom_name = parts[1]
            x, y, z = map(float, parts[2:5])
            atom_type = parts[5]
            subst_id = int(parts[6])
            subst_name = parts[7]
            charge = float(parts[8])
            self.atoms.append(self.Atom(atom_id, atom_name, x, y, z, atom_type, subst_id, subst_name, charge))

    def parse_bonds(self, lines):
        for line in lines:
            parts = re.split(r'\s+', line.strip())
            bond_id = int(parts[0])
            origin_atom_id, target_atom_id = map(int, parts[1:3])
            bond_type = int(parts[3])
            self.bonds.append(self.Bond(bond_id, origin_atom_id, target_atom_id, bond_type))

    def renumber(self):
        atom_id_mapping = {}

        for i, atom in enumerate(self.atoms, 1):
            atom_id_mapping[atom.atom_id] = i
            atom.atom_id = i

        for bond in self.bonds:
            bond.origin_atom_id = atom_id_mapping[bond.origin_atom_id]
            bond.target_atom_id = atom_id_mapping[bond.target_atom_id]

        for i, bond in enumerate(self.bonds, 1):
            bond.bond_id = i

    def __str__(self):
        output = []

        # Molecule section
        output.append("@<TRIPOS>MOLECULE")
        output.append(self.molecule_info["name"])
        output.append(f"{self.molecule_info['num_atoms']} {self.molecule_info['num_bonds']} 0 0 0")
        output.append(self.molecule_info["type"])
        output.append(self.molecule_info["charge_method"])
        output.append("")

        # Atom section
        output.append("@<TRIPOS>ATOM")
        for atom in self.atoms:
            # output.append(f"{atom.atom_id}\t{atom.atom_name}\t{atom.x:.4f}\t{atom.y:.4f}\t{atom.z:.4f}\t{atom.atom_type}\t{atom.subst_id}\t{atom.subst_name}\t {atom.charge:.4f}")
            output.append(
                f"{atom.atom_id}\t{atom.atom_name}\t    ".expandtabs(4) +
                f"{atom.x:.4f}\t{atom.y:.4f}\t{atom.z:.4f}\t".expandtabs(9) + 
                f"{atom.atom_type}\t".expandtabs(7) + 
                f"{atom.subst_id}\t".expandtabs(4) + 
                f"{atom.subst_name}\t {atom.charge:.4f}".expandtabs(8)
                )

        # Bond section
        output.append("@<TRIPOS>BOND")
        for bond in self.bonds:
            output.append(f"{bond.bond_id}\t{bond.origin_atom_id}\t{bond.target_atom_id}\t{bond.bond_type}".expandtabs(4))

        return "\n".join(output).expandtabs(8)
    
    def write_file(self, filename):
        with open(filename, 'w') as f:
            f.write(str(self))

class XYZ_file:

    class Atom:
        def __init__(self, atom_name, x, y, z):
            self.atom_name = atom_name
            self.x = x
            self.y = y
            self.z = z

        def __repr__(self):
            return f"Atom({self.atom_name}, {self.x}, {self.y}, {self.z})"


    def __init__(self, atoms=None, comment="", molID=""):
        """Initialize an XYZ object.
        
        Parameters:
            atoms (list): List of Atom instances.
            comment (str): Comment or description for the XYZ data.
        """
        self.atoms = atoms if atoms else []
        self.comment = comment
        self.molID = molID

    @classmethod
    def from_file(cls, filepath):
        """Load an XYZ object from a file."""
        with open(filepath, 'r') as file:
            lines = file.readlines()
            
            num_atoms = int(lines[0].strip())
            comment = lines[1].strip()
            
            atoms = []
            for line in lines[2:2+num_atoms]:
                parts = line.split()
                atom_name = parts[0]
                x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
                atoms.append(cls.Atom(atom_name, x, y, z))

            molID = os.path.basename(filepath).split('.')[0]
            
            return cls(atoms, comment, molID)

    def to_file(self, filepath):
        """Write the XYZ data to a file."""
        with open(filepath, 'w') as file:
            file.write(self.__str__())

    def __str__(self):
        result = []
        result.append(str(len(self.atoms)))
        result.append(self.comment)
        for atom in self.atoms:
            result.append(f'{atom.atom_name:<4}  {atom.x:>15.10f}  {atom.y:>15.10f}  {atom.z:>15.10f}')
        return "\n".join(result)

    def __repr__(self):
        return f"XYZ({self.atoms}, '{self.comment}')"


In [3]:
def abs_path(filename=None, extra_dir=''):
    output_directory = os.path.join(current_dir, 'out', extra_dir)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    if filename is None: return output_directory
    return os.path.join(output_directory, filename)

In [4]:
def is_metal(atomic_number):
    """Return True if atomic_number is the atomic number of a metal."""
    # TODO 此方法需要改进，现在只适用于 Cr
    return atomic_number in [24,]

def is_metal_str(atom_name):
    """Return True if atom_name is the name of a metal."""
    # TODO 这是为了编过号的mol2文件制作的判定方法。同上，这个方法需要改进
    atom_name = re.sub(r'\d', '', atom_name).capitalize()
    if atom_name in ['Cr']: return True
    return False

In [5]:
test_mol_list = [
    MolID194,
    MolID239,
    ]

# 电荷分配部分

In [6]:
def create_mol(MolID):
    Molxyz = XYZ_file.from_file(os.path.join(current_dir, MolID+'.xyz'))
    Mol_pybel = pybel.readstring('xyz', str(Molxyz))
    return Mol_pybel

M_pybel_dict = {} # TODO 之后这个字典要移除，一次只处理一个分子，减少不必要存储空间占用
for M_ID in test_mol_list:
    M_pybel_dict[M_ID] = create_mol(M_ID)

In [7]:
# 将分子的电荷进行重新分配
def reallocate_charges(M_pybel, MolCharge, printFLag=False):
    """
    M_pybel: pybel.Molecule
    MolCharge: float
    """

    def find_metal_idx(M_OBMol):
        """Find the index of the metal atom in the molecule."""

        # metal_atom_list = []
        for idx in range(1, M_OBMol.NumAtoms()+1):
            atom = M_OBMol.GetAtom(idx)
            if is_metal(atom.GetAtomicNum()): 
                return idx
            # metal_atom_list.append(idx)
        # return metal_atom_list

    def set_fcharge(M_OBMol, MolCharge):

        metal_idx = find_metal_idx(M_OBMol)
        metal_atom = M_OBMol.GetAtom(metal_idx)

        # Temporarily set the charge of the metal atom to the molecule charge
        metal_atom.SetFormalCharge(MolCharge)

        # 然后找到所有与金属相连的原子，断键，设置金属电荷+1，配体电荷-1（如果为多重键则电荷值为键级）
        neigbor_bonds = [] # [(bond, bond_order, nbr_atom_idx)]
        for nbr_bond in ob.OBAtomBondIter(metal_atom):
            neigbor_bonds.append((nbr_bond, nbr_bond.GetBondOrder(), nbr_bond.GetNbrAtom(metal_atom).GetIdx()))

        for bond, bond_order, nbr_atom_idx in neigbor_bonds:
            nbr_atom = M_OBMol.GetAtom(nbr_atom_idx)
            nbr_atom.SetFormalCharge(-bond_order)
            # print(f'Metal_atom_charge: currently {metal_atom.GetFormalCharge()}, set to {metal_atom.GetFormalCharge()+bond_order}')
            metal_atom.SetFormalCharge(metal_atom.GetFormalCharge()+bond_order)
            M_OBMol.DeleteBond(bond)

    def set_pcharge_to_fcharge(M_OBMol):
        for atom in ob.OBMolAtomIter(M_OBMol):
            atom_fcharge = atom.GetFormalCharge()
            atom.SetPartialCharge(atom_fcharge)
            if printFLag and atom.GetIdx() == 1: print(f'atom index {atom.GetIdx()} set pcharge to {atom.GetPartialCharge()} (atom fcharge is {atom.GetFormalCharge()})')

    M_OBMol = M_pybel.OBMol
    M_OBMol.SetAutomaticPartialCharge(False)
    set_fcharge(M_OBMol, MolCharge)
    set_pcharge_to_fcharge(M_OBMol)
    if printFLag: print('------------------')
    set_pcharge_to_fcharge(M_OBMol) # 如果不这样运行两次会发生很神奇的错误现象，原因不明（两次运行结果不一致，如打印所示）

for M_ID in test_mol_list:
    reallocate_charges(M_pybel_dict[M_ID], MolCharge194, printFLag=True)
    M_pybel_dict[M_ID].write('mol2', abs_path(f'{M_ID}-pybel.mol2', extra_dir=f'{M_ID}'), overwrite=True)

# M194_pybel.write('mol2', abs_path(f'{MolID194}-pybel.mol2'), overwrite=True)
# M239_pybel.write('mol2', abs_path(f'{MolID239}-pybel.mol2'), overwrite=True)

atom index 1 set pcharge to 3.0 (atom fcharge is 3)
------------------
atom index 1 set pcharge to 3.0 (atom fcharge is 3)
atom index 1 set pcharge to -3.0 (atom fcharge is -3)
------------------
atom index 1 set pcharge to -3.0 (atom fcharge is -3)


# 金属原子拆分部分

接下来将会脱离OBMol对象，转为纯文件处理

In [8]:
def split_mol2(M_ID):

    def readin_mol2(M_ID):
        with open(abs_path(f'{M_ID}-pybel.mol2', extra_dir=f'{M_ID}'), 'r') as f:
            M_mol2 = Mol2_file(f.read())
        return M_mol2

    def rename_atoms_in_mol2_object(mol2_obj):
        # Dictionary to store count of atom types
        atom_count = {}
        for atom in mol2_obj.atoms:
            atom_type = atom.atom_name
            atom_count[atom_type] = atom_count.get(atom_type, 0) + 1
            atom.atom_name = f"{atom_type}{atom_count[atom_type]}"
        return mol2_obj

    def split_mol2_by_metal(original_mol2):

        metal_mol2 = Mol2_file("")
        non_metal_mol2 = Mol2_file("")
        
        metal_mol2.molecule_info = original_mol2.molecule_info.copy()
        non_metal_mol2.molecule_info = original_mol2.molecule_info.copy()
        
        metal_mol2.molecule_info["num_atoms"] = 0
        metal_mol2.molecule_info["num_bonds"] = 0
        
        for atom in original_mol2.atoms:
            if is_metal_str(atom.atom_name):
                metal_mol2.atoms.append(atom)
                metal_mol2.molecule_info["num_atoms"] += 1
            else:
                non_metal_mol2.atoms.append(atom)

        non_metal_mol2.molecule_info["num_atoms"] = original_mol2.molecule_info["num_atoms"] - metal_mol2.molecule_info["num_atoms"]
        non_metal_mol2.bonds = original_mol2.bonds
        non_metal_mol2.molecule_info["num_bonds"] = len(original_mol2.bonds)
        
        return metal_mol2, non_metal_mol2

    def split_non_metal_mol2(M_non_metal_mol2):

        from collections import deque

        def assign_molecules(mol2_obj):
            num_atoms = mol2_obj.molecule_info["num_atoms"]
            num_bonds = mol2_obj.molecule_info["num_bonds"]

            atom_assignments = [0] * num_atoms
            bond_assignments = [0] * num_bonds
            
            molecule_id = 1
            
            while 0 in atom_assignments:
                # 找到第一个未被归属的原子
                current_atom_idx = atom_assignments.index(0)
                atom_assignments[current_atom_idx] = molecule_id
                
                # 使用队列来记录待检查的原子
                queue = deque([mol2_obj.atoms[current_atom_idx].atom_id])
                
                while queue:
                    atom_id = queue.popleft()
                    # 遍历所有键，查找与当前原子连接的其他原子
                    for i, bond in enumerate(mol2_obj.bonds):
                        if bond_assignments[i] == 0:  # 只考虑未被归属的键
                            if bond.origin_atom_id == atom_id or bond.target_atom_id == atom_id:
                                bond_assignments[i] = molecule_id  # 更新键的归属
                                
                                # 将与当前原子连接的其他原子加入到队列中
                                if bond.origin_atom_id == atom_id:
                                    neighbor_atom_id = bond.target_atom_id
                                else:
                                    neighbor_atom_id = bond.origin_atom_id
                                
                                neighbor_atom_idx = next(index for index, atom in enumerate(mol2_obj.atoms) if atom.atom_id == neighbor_atom_id)
                                if atom_assignments[neighbor_atom_idx] == 0:
                                    atom_assignments[neighbor_atom_idx] = molecule_id
                                    queue.append(neighbor_atom_id)
                
                molecule_id += 1
            
            return atom_assignments, bond_assignments

        def split_mol2_file(mol2_obj, atom_assignments, bond_assignments):
            num_molecules = max(atom_assignments)

            mol2_objects = []
            
            for i in range(1, num_molecules + 1):
                molecule_atoms = [atom for idx, atom in enumerate(mol2_obj.atoms) if atom_assignments[idx] == i]
                molecule_bonds = [bond for idx, bond in enumerate(mol2_obj.bonds) if bond_assignments[idx] == i]

                new_mol2_obj = Mol2_file("")
                new_mol2_obj.atoms = molecule_atoms
                new_mol2_obj.bonds = molecule_bonds

                new_mol2_obj.molecule_info = mol2_obj.molecule_info.copy()
                new_mol2_obj.molecule_info["num_atoms"] = len(molecule_atoms)
                new_mol2_obj.molecule_info["num_bonds"] = len(molecule_bonds)

                mol2_objects.append(new_mol2_obj)
            
            return mol2_objects

        return split_mol2_file(M_non_metal_mol2, *assign_molecules(M_non_metal_mol2))

    M_mol2 = rename_atoms_in_mol2_object(readin_mol2(M_ID))
    M_mol2.write_file(abs_path(f'{M_ID}-renamed.mol2', extra_dir=f'{M_ID}'))

    M_metal_mol2, M_non_metal_mol2 = split_mol2_by_metal(M_mol2)
    M_metal_mol2.write_file(abs_path(f'{M_ID}-metal.mol2', extra_dir=f'{M_ID}'))

    M_legands_mol2_list = split_non_metal_mol2(M_non_metal_mol2)
    for idx, splitted_mol2_obj in enumerate(M_legands_mol2_list):
        splitted_mol2_obj.renumber()
        splitted_mol2_obj.write_file(abs_path(f'{M_ID}-legand{idx+1}.mol2', extra_dir=f'{M_ID}'))

for M_ID in test_mol_list:
    split_mol2(M_ID)

# 准备计算所需文件

In [13]:
M_ID = MolID194

In [14]:
# 制作分子pdb
input_file, output_file = abs_path(f'{M_ID}-renamed.mol2', extra_dir=f'{M_ID}'), abs_path('MOL.pdb', extra_dir=f'{M_ID}')
subprocess.run(["obabel", input_file, "-O", output_file], check=True)

1 molecule converted


CompletedProcess(args=['obabel', '/Users/suyian/Desktop/Inbox/SumIntern/WS-0825/out/194-1/194-1-renamed.mol2', '-O', '/Users/suyian/Desktop/Inbox/SumIntern/WS-0825/out/194-1/MOL.pdb'], returncode=0)

In [15]:
# 处理金属mol2
input_file, output_file = abs_path(f'{M_ID}-metal.mol2', extra_dir=f'{M_ID}'), abs_path('MET.mol2', extra_dir=f'{M_ID}')
subprocess.run(["mv", input_file, output_file])

CompletedProcess(args=['mv', '/Users/suyian/Desktop/Inbox/SumIntern/WS-0825/out/194-1/194-1-metal.mol2', '/Users/suyian/Desktop/Inbox/SumIntern/WS-0825/out/194-1/MET.mol2'], returncode=0)

In [16]:
# 处理配体
import glob

source_pattern = f'{M_ID}-legand*.mol2'
files_to_move = glob.glob(abs_path(source_pattern, extra_dir=f'{M_ID}'))

for file in files_to_move:
    suffix = file.rsplit('.', maxsplit=1)[0].rsplit('-', maxsplit=1)[-1].replace('legand', '')
    #new_name = f"LIG{chr(64+int(suffix))}.mol2"
    new_file = f"LIG{suffix}.mol2"
    subprocess.run(["mv", file, abs_path(new_file, extra_dir=f'{M_ID}')], check=True)
    #subprocess.run(["antechamber", "-i", file, "-fi", "mol2", "-o", new_file, "-fo", "mol2", "-c", "bcc", "-pf", "yes"])
    subprocess.run(["parmchk2", "-i", new_file, "-o", f"LIG{suffix}.frcmod", "-f", "mol2"], cwd=abs_path(extra_dir=f'{M_ID}'))

# LOG: antechamber: 无法处理三键
# parmchk2: 无法处理常见配体中的原子，如三配位氧、CN中的N等

Atom type of N.1 does not exist in PARMCHK.DAT
Atom type of N.1 does not exist in PARMCHK.DAT
Atom type of N.1 does not exist in PARMCHK.DAT


Atom type of N.1 does not exist in PARMCHK.DAT
Atom type of N.1 does not exist in PARMCHK.DAT
Atom type of N.1 does not exist in PARMCHK.DAT
